This notebook creates a folder in `base_target_path` (TODO: update this one when running this nb anew!) with the stimuli for the practice trials in the structure required for the server.

`trial_[1-3]` are very easy; `trial_[4-10]` are easy. This was judged by Roland and Judy.

# Imports

In [ ]:
import os
import glob
import shutil
import random

# Parameters

In [ ]:
# TODO: update these two folders
base_source_path = "$DATAPATH/stimuli_generated_on_20210512"
base_target_path = "$DATAPATH/stimuli_practice_generated_on_20210513" 
os.makedirs(base_target_path, exist_ok=True)

source_path_pure = os.path.join(
    base_source_path,
    "stimuli_pure_conditions/channel/instruction_practice_catch"
)
source_path_mixed = os.path.join(
    base_source_path,
    "stimuli_mixed_conditions/channel/instruction_practice_catch"
)

In [ ]:
# This dictionary was generated based on this Google Doc:
# https://docs.google.com/document/d/1eYjyAzJDm-v8Z0D3wS70lEP_dczze9CoPYMwodJVpjo/edit?usp=sharing

practice_info_dict = {}
practice_info_dict[1]= {
    "layer_number": "g",
    "batch_number": 18
}
practice_info_dict[2]= {
    "layer_number": "o",
    "batch_number": 1
}
practice_info_dict[3]= {
    "layer_number": "d",
    "batch_number": 0
}
practice_info_dict[4]= {
    "layer_number": "c",
    "batch_number": 5
}
practice_info_dict[5]= {
    "layer_number": "e",
    "batch_number": 2
}
practice_info_dict[6]= {
    "layer_number": "f",
    "batch_number": 12
}
practice_info_dict[7]= {
    "layer_number": "h",
    "batch_number": 0
}
practice_info_dict[8]= {
    "layer_number": "k",
    "batch_number": 9
}
practice_info_dict[9]= {
    "layer_number": "n",
    "batch_number": 3
}
practice_info_dict[10]= {
    "layer_number": "q",
    "batch_number": 5
}

In [ ]:
def get_source_path_base(mixed_or_pure):
    if mixed_or_pure == "mixed":
        source_path_base = source_path_mixed
    elif mixed_or_pure == "pure":
        source_path_base = source_path_pure

    return source_path_base

# Query Images

In [ ]:
def copy_query_imgs(mixed_or_pure, target_folder):

    for trial_i, feature_map_specification in practice_info_dict.items():

        source_path_base = get_source_path_base(mixed_or_pure)
        source_queries_path = os.path.join(
            source_path_base, 
            f"layer_{feature_map_specification['layer_number']}",
            f"kernel_size_{feature_map_specification['layer_number']}",
            f"channel_{feature_map_specification['layer_number']}",
            "natural_images",
            f"batch_{feature_map_specification['batch_number']}",
            "40_percent_side_length")

        target_queries_path = os.path.join(
            target_folder,
            f"trial_{trial_i}",
            "queries"
        )
        os.makedirs(target_queries_path, exist_ok=True)

        shutil.copy(os.path.join(source_queries_path, "query_min_activation.png"), os.path.join(target_queries_path, "min.png"))
        shutil.copy(os.path.join(source_queries_path, "query_max_activation.png"), os.path.join(target_queries_path, "max.png"))
        shutil.copy(os.path.join(source_queries_path, "query_default.png"), os.path.join(target_queries_path, "base.png"))

In [ ]:
for mixed_or_pure, target_folder_base in zip(
    ["pure", "pure", "pure", "mixed"], 
    ["demo_natural_9_references",
     "demo_natural_blur_9_references",
     "demo_optimized_9_references", 
     "demo_natural_5_optimized_4_references"
    ]):
        target_folder = os.path.join(
            base_target_path, 
            target_folder_base, 
            "trials"
        )
        os.makedirs(target_folder, exist_ok=True)
        
        copy_query_imgs(mixed_or_pure, target_folder)

# Reference Images

In [ ]:
def copy_reference_imgs(mixed_or_pure, target_folder):
    for trial_i, feature_map_specification in practice_info_dict.items():

        source_path_base = get_source_path_base(mixed_or_pure)
        source_queries_path_base = os.path.join(
            source_path_base, 
            f"layer_{feature_map_specification['layer_number']}",
            f"kernel_size_{feature_map_specification['layer_number']}",
            f"channel_{feature_map_specification['layer_number']}"
        )
        
        target_references_path = os.path.join(
            target_folder,
            f"trial_{trial_i}",
            "references"
        )
        os.makedirs(target_references_path, exist_ok=True)

        if mixed_or_pure == "pure":
            if "natural_blur" in target_folder:
                source_queries_path_condition = os.path.join(
                    "natural_blur_images",
                    f"batch_{feature_map_specification['batch_number']}",
                    "40_percent_side_length"
                )
            elif "natural" in target_folder:
                source_queries_path_condition = os.path.join(
                    "natural_images",
                    f"batch_{feature_map_specification['batch_number']}"
                )
            elif "optimized" in target_folder:
                source_queries_path_condition = "optimized_images"
            imgs_paths_list = glob.glob(
                os.path.join(
                    source_queries_path_base,
                    source_queries_path_condition,
                    "*.png"
                )
            )

            # first copy over the base image as the very last image in the folder
            last_reference = imgs_paths_list[-1] # depending on file system, not necessarily the max_5 img!
            base_img_dirname = os.path.dirname(last_reference)
            last_img_basename = os.path.basename(last_reference)
            base_img_basename = f"{last_img_basename.split('.png')[0][:-1]}{len(imgs_paths_list)-1}.png"
            source_reference = os.path.join(base_img_dirname, base_img_basename)
            target_reference = os.path.join(target_references_path, f"reference_max_{len(imgs_paths_list) -1}.png")
            shutil.copy(source_reference, target_reference)
            # remove path to base img from list
            imgs_paths_list.remove(source_reference)

            # copy remaining images over
            # shuffle the image numbering                      
            random.shuffle(imgs_paths_list)
            for reference_idx, source_reference in enumerate(imgs_paths_list):
                target_reference = os.path.join(target_references_path, f"reference_max_{reference_idx}.png")
                shutil.copy(source_reference, target_reference)

        elif mixed_or_pure == "mixed":
            optimized_imgs_paths_list = imgs_paths_list = glob.glob(
                os.path.join(
                    source_queries_path_base,
                    "optimized_images",
                    "*.png"
                )
            )
            natural_imgs_paths_list = glob.glob(
                os.path.join(
                    source_queries_path_base,
                    "natural_images",
                    f"batch_{feature_map_specification['batch_number']}",
                    "*.png"
                )
            )            

            # first copy over the natural base image as the very last image in the mixed folder
            last_reference = natural_imgs_paths_list[-1] # depending on file system, not necessarily the max_5 img!
            base_img_dirname = os.path.dirname(last_reference)
            last_img_basename = os.path.basename(last_reference)
            base_img_basename = f"{last_img_basename.split('.png')[0][:-1]}{len(natural_imgs_paths_list)-1}.png"
            source_reference = os.path.join(base_img_dirname, base_img_basename)
            target_reference = os.path.join(target_references_path, f"reference_max_{len(natural_imgs_paths_list) + len(optimized_imgs_paths_list) -1}.png")
            shutil.copy(source_reference, target_reference)
            # remove path to base img from list
            natural_imgs_paths_list.remove(source_reference)

            # shuffle the image numbering
            random.shuffle(optimized_imgs_paths_list)
            random.shuffle(natural_imgs_paths_list)
            # first copy optimized (indices 0-3) and then optimized images (indices 4-8)
            for reference_idx, source_reference in enumerate(optimized_imgs_paths_list):
                # continue counting of reference images
                target_reference = os.path.join(target_references_path,
                                                f"reference_max_{reference_idx}.png")
                shutil.copy(source_reference, target_reference)   
            for reference_idx, source_reference in enumerate(natural_imgs_paths_list):
                target_reference = os.path.join(target_references_path, 
                                                f"reference_max_{reference_idx + len(optimized_imgs_paths_list)}.png")
                shutil.copy(source_reference, target_reference)

In [ ]:
for mixed_or_pure, target_folder_base in zip(
    ["pure", "pure", "pure", "mixed"], 
    ["demo_natural_9_references", 
     "demo_natural_blur_9_references",
     "demo_optimized_9_references", 
     "demo_natural_5_optimized_4_references"
    ]):
        target_folder = os.path.join(
            base_target_path, 
            target_folder_base, 
            "trials")
        os.makedirs(target_folder, exist_ok=True)
        print(target_folder_base)
        
        copy_reference_imgs(mixed_or_pure, target_folder)